# Data Preprocessing for Vision tasks

One of the most important step in computer vision tasks is data preprocessing. Once we got the dataset, we need to make sure that our dataset matches our requirement. Main reasons for preprocessing are to improve data quality, improve model quality and to make the data shape.

To do the preprocessing operations we can use the tensor library or keras api (Personally I think keras API makes more sense to use in preprocessing steps as it does not change very often).

The advantage of using such preprocessing layer is that then it becomes a part of the ML model. Therefore we do not need to make processing pipelines to handle data during inferencing phase. 

## Data augmenting

Another application of data preprocessing is to make augmented datasets. In many ML applications it is hard to find a large enough dataset for our work. Also sometimes it is better to train our model in different variations of same image for more robust outputs and data augmentation is a way to achieve that.

### Spatial Transformations

Under this technique we do random cropping, zooming, cropping flipping, rotation etc. This transformation makes the dataset larger by providing more varieties of same types of image.

But depending on the usecase this transformation need to be chose wisely. For example if we flipped an image vertically it may not be a useful data point for a ML model. Keras api provides several augmentation layers including random translation, rotation, zoom, flip and so on.



### Color Distortion

Another main type of augmentation we can perform is color change. For example in cases of images taken by amatuers there maybe cases of overexposures and underexposures compared to professional images. Therefore in order to train the model to compensate for such scenarios we can augment our training datasets with color distorted samples.

We can implement a simple keras layer for color distortion like below. Or can use the provided class for the same as well.

Point to note is that, this type of layers should behave differently during the training and testing phases. We dont want our actual images to be randomly changed during the model inference!

In [ ]:
import tensorflow as tf
import numpy as np


class RandomColorDistortion(tf.keras.layers.Layer):

    def __init__(self, contrast_range=[0.5, 1.5], brightness_range=[-0.2, 0.2], **kwargs):
        super(RandomColorDistortion, self).__init__(**kwargs)

        self.contrast_range=contrast_range
        self.brightness_range=brightness_range

    
    def call(self, images, training=False):

        if not training:
            return images

        contrast = np.random.uniform(self.contrast_range[0], self.contrast_range[1])
        brightness = np.random.uniform(self.brightness_range[0], self.brightness_range[1])

        images = tf.image.adjust_contrast(images, contrast)
        images = tf.image.adjust_brightness(images, brightness)
        images = tf.clip_by_value(images, 0, 1)

        return images

### Information Dropping

Based on the various experimentations done on data augmentation, researchers realized the technique of dropping information makes model more resillient and model to identify important features better.

Below are such data regularization techniques used.

- Cutout --> Randomly mask out parts of input data during the training process.

- Mixup --> Interpolate pair of training images and assign their label the corresponding interpolated label value. (not suitable for many computer vision use cases)

- Cutmix --> Combination of above 2 techniques.

- GridMask --> Randomly mask out regions of image from a grid (assumption is that grid might include background segments rather than object of interest)


### Forming Input Images

The idea behind this technique is to break down single image to multiple types which can then be used as inputs to a ML model. 

One such method is called `Tiling`. This is useful in situations where we have extremely large images but can carry out our work on smaller segments which then can be combined later. For example geospatial imagery, medical images are some examples.

Based on our usecase we can combine/assemble images to get the desired output.